In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, f1_score

bikesDf = pd.read_csv("day.csv")

X = bikesDf.drop(columns=['cnt', 'dteday'])  
y = bikesDf['cnt']  # Target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

explained_variance_ratio = pca.explained_variance_ratio_

cumulative_explained_variance = explained_variance_ratio.cumsum()

rental_bins = pd.qcut(y, q=3, labels=['low', 'medium', 'high'])
X_train, X_test, y_train, y_test = train_test_split(X_pca, rental_bins, test_size=0.2, random_state=42)

svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("PCA Results:")
print("Explained Variance Ratio:")
for i, ratio in enumerate(explained_variance_ratio):
    print(f"Component {i+1}: {ratio:.4f}")

print("\nCumulative Explained Variance:")
for i, cm_ratio in enumerate(cumulative_explained_variance):
    print(f"Up to Component {i+1}: {cm_ratio:.4f}")

print("\nConfusion Matrix:")
print(conf_matrix)
print("\nF1 Score:", f1)


PCA Results:
Explained Variance Ratio:
Component 1: 0.2723
Component 2: 0.1423
Component 3: 0.1309
Component 4: 0.1088
Component 5: 0.0872
Component 6: 0.0748
Component 7: 0.0659
Component 8: 0.0588
Component 9: 0.0228
Component 10: 0.0171
Component 11: 0.0117
Component 12: 0.0068
Component 13: 0.0006
Component 14: 0.0001

Cumulative Explained Variance:
Up to Component 1: 0.2723
Up to Component 2: 0.4146
Up to Component 3: 0.5456
Up to Component 4: 0.6543
Up to Component 5: 0.7415
Up to Component 6: 0.8163
Up to Component 7: 0.8822
Up to Component 8: 0.9410
Up to Component 9: 0.9638
Up to Component 10: 0.9809
Up to Component 11: 0.9925
Up to Component 12: 0.9994
Up to Component 13: 0.9999
Up to Component 14: 1.0000

Confusion Matrix:
[[43  0  4]
 [ 0 55  5]
 [ 4  4 32]]

F1 Score: 0.8848103615343845


In [3]:
print(bikesDf.describe())
print(bikesDf.columns.tolist())

          instant      season          yr        mnth     holiday     weekday   
count  731.000000  731.000000  731.000000  731.000000  731.000000  731.000000  \
mean   366.000000    2.496580    0.500684    6.519836    0.028728    2.997264   
std    211.165812    1.110807    0.500342    3.451913    0.167155    2.004787   
min      1.000000    1.000000    0.000000    1.000000    0.000000    0.000000   
25%    183.500000    2.000000    0.000000    4.000000    0.000000    1.000000   
50%    366.000000    3.000000    1.000000    7.000000    0.000000    3.000000   
75%    548.500000    3.000000    1.000000   10.000000    0.000000    5.000000   
max    731.000000    4.000000    1.000000   12.000000    1.000000    6.000000   

       workingday  weathersit        temp       atemp         hum   windspeed   
count  731.000000  731.000000  731.000000  731.000000  731.000000  731.000000  \
mean     0.683995    1.395349    0.495385    0.474354    0.627894    0.190486   
std      0.465233    0.5448

In [4]:
# Perform PCA before splitting the data
pca = PCA(n_components=10)  # Choose the number of components
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.transform(X_test)

# Train SVM on the PCA-transformed training data
svm = SVC()
svm.fit(X_pca_train, y_train)

# Predict labels for the transformed test data
y_pred = svm.predict(X_pca_test)

# Evaluate the performance of the classifier
conf_matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("\nPCA + SVM Results:")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nF1 Score:", f1)



PCA + SVM Results:

Confusion Matrix:
[[43  0  4]
 [ 0 54  6]
 [ 4  4 32]]

F1 Score: 0.8784677988599325


In [5]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10]}

# Initialize GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_pca_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("\nBest Parameters:", best_params)
print("Best Score:", best_score)

# Predict labels for the test data using the best estimator
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_pca_test)

# Evaluate the performance of the classifier
conf_matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("\nPCA + Optimized SVM Results:")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nF1 Score:", f1)



Best Parameters: {'C': 100, 'gamma': 0.01}
Best Score: 0.9331859711170056

PCA + Optimized SVM Results:

Confusion Matrix:
[[46  0  1]
 [ 0 56  4]
 [ 3  1 36]]

F1 Score: 0.9390008140008143
